
## Import 

In [6]:
cd ..

/home/alex/Desktop/openclassroom/mentorat/__alexandre_gazagnes_mentorat/DS/P7


In [7]:
import os, sys, pickle

from IPython.display import display

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
from xgboost import XGBClassifier



# import shap

## Load data

In [8]:
url = "./data/cleaned/df.csv"

In [9]:
df = pd.read_csv (url)

FileNotFoundError: [Errno 2] No such file or directory: './data/cleaned/df.csv'

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


## Prepare

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [10]:
X = df.drop(columns="Survived")
y = df.Survived

In [11]:
df_train, df_test  = train_test_split(df)

In [12]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
739,740,0,3,male,NaN,0,0,7.8958,S
175,176,0,3,male,18.0,1,1,7.8542,S
177,178,0,1,female,50.0,0,0,28.7125,C
835,836,1,1,female,39.0,1,1,83.1583,C
772,773,0,2,female,57.0,0,0,10.5000,S


In [13]:
df_train.shape

(668, 9)

In [17]:
df_test.shape

(223, 9)

In [20]:
y_train = df_train.Survived.values

In [21]:
# X_train_PassengerId = df_train.PassengerId.values

X_train = df_train.drop(columns=["Survived", ], inplace=False)
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
739,740,3,male,NaN,0,0,7.8958,S
175,176,3,male,18.0,1,1,7.8542,S
177,178,1,female,50.0,0,0,28.7125,C
835,836,1,female,39.0,1,1,83.1583,C
772,773,2,female,57.0,0,0,10.5000,S


In [22]:
y_train[:10]

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0])

In [23]:
len(y_train)

668

In [24]:
len(X_train)

668

In [ ]:
y_test = df_test.Survived.values


In [26]:
# X_test_PassengerId = df_test.PassengerId.values
X_test = df_test.drop(columns=["Survived" ], inplace=False)
X_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
464,465,3,male,NaN,0,0,8.0500,S
870,871,3,male,26.0,0,0,7.8958,S
715,716,3,male,19.0,0,0,7.6500,S
459,460,3,male,NaN,0,0,7.7500,Q
332,333,1,male,38.0,0,1,153.4625,S


In [36]:
X_train.select_dtypes(include=np.number).columns

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [38]:
X_train.select_dtypes(exclude=np.number).columns

Index(['Sex', 'Embarked'], dtype='object')

## Model

In [57]:
preprocessor = ColumnTransformer(transformers=[
    ("num", "passthrough",['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
    ("oneHot", OneHotEncoder( handle_unknown="ignore"), ['Sex', 'Embarked'])
    ], remainder="drop")

In [58]:
X_train.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
739,740,3,male,NaN,0,0,7.8958,S
175,176,3,male,18.0,1,1,7.8542,S
177,178,1,female,50.0,0,0,28.7125,C
835,836,1,female,39.0,1,1,83.1583,C
772,773,2,female,57.0,0,0,10.5000,S
457,458,1,female,NaN,1,0,51.8625,S
351,352,1,male,NaN,0,0,35.0000,S
682,683,3,male,20.0,0,0,9.2250,S
553,554,3,male,22.0,0,0,7.2250,C
14,15,3,female,14.0,0,0,7.8542,S


In [59]:
tmp = preprocessor.fit_transform(X_train)
pd.DataFrame(tmp).head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,3.0,NaN,0.0,0.0,7.8958,0.0,1.0,0.0,0.0,1.0,0.0
1,3.0,18.0,1.0,1.0,7.8542,0.0,1.0,0.0,0.0,1.0,0.0
2,1.0,50.0,0.0,0.0,28.7125,1.0,0.0,1.0,0.0,0.0,0.0
3,1.0,39.0,1.0,1.0,83.1583,1.0,0.0,1.0,0.0,0.0,0.0
4,2.0,57.0,0.0,0.0,10.5000,1.0,0.0,0.0,0.0,1.0,0.0
5,1.0,NaN,1.0,0.0,51.8625,1.0,0.0,0.0,0.0,1.0,0.0
6,1.0,NaN,0.0,0.0,35.0000,0.0,1.0,0.0,0.0,1.0,0.0
7,3.0,20.0,0.0,0.0,9.2250,0.0,1.0,0.0,0.0,1.0,0.0
8,3.0,22.0,0.0,0.0,7.2250,0.0,1.0,1.0,0.0,0.0,0.0
9,3.0,14.0,0.0,0.0,7.8542,1.0,0.0,0.0,0.0,1.0,0.0


In [60]:
imputer = KNNImputer()

# X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
# X_test = pd.DataFrame(imputer.transform(X_test), columns=X_train.columns)
# X_train.info()

In [61]:
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("imputer",imputer),
    ("estimator", RandomForestClassifier())
])
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('oneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Embarked'])])),
                ('imputer', KNNImputer()),
                ('estimator', RandomForestClassifier())])

In [62]:
grid = GridSearchCV(pipe, {}, cv=10, return_train_score=True, n_jobs=-1)

## Train

In [63]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         'passthrough',
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('oneHot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Sex',
                                                                          'Embarked'])])),
                                       ('imputer', KNNImputer()),
                                       ('estimator',
                                        RandomForestClassifier())]),
             n_jobs=-1, param_grid={}, return_train_score=True)

In [65]:
res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if ("split" not in i) ]
res = res.loc[:, cols]
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,0.290441,0.067716,0.019874,0.004673,{},0.817368,0.031925,1,0.984697,0.002208


## Test

In [66]:
grid.score(X_train, y_train)

0.9835329341317365

In [68]:
model = grid.best_estimator_
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('oneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Embarked'])])),
                ('imputer', KNNImputer()),
                ('estimator', RandomForestClassifier())])

In [69]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('oneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Embarked'])])),
                ('imputer', KNNImputer()),
                ('estimator', RandomForestClassifier())])

In [71]:
model.score(X_test, y_test)

NameError: name 'y_test' is not defined

In [72]:
pre = grid.best_estimator_['preprocessor']
pre


ColumnTransformer(transformers=[('num', 'passthrough',
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
                                ('oneHot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Sex', 'Embarked'])])

In [74]:
imp = pre = grid.best_estimator_['imputer']
imp

KNNImputer()

In [76]:
est = grid.best_estimator_['estimator']
est

RandomForestClassifier()

# export

In [80]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(model)
    f.write(pk)

In [81]:
with open("./models/preprocessor.pk", "wb") as f : 
    pk = pickle.dumps(pre)
    f.write(pk)

In [83]:
with open("./models/imputer.pk", "wb") as f : 
    pk = pickle.dumps(imp)
    f.write(pk)

In [84]:
with open("./models/estimator.pk", "wb") as f : 
    pk = pickle.dumps(est)
    f.write(pk)

In [26]:
display(len(X_train))
display(len(X_train_PassengerId))
display(len(y_train))

668

668

668

In [27]:
len(y_train)

668

In [28]:
X_train["Survived"] = y_train
X_train["PassengerId"] = X_train_PassengerId
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived,PassengerId
0,3.0,35.9,0.0,0.0,7.7500,0.0,1.0,0.0,1.0,0.0,0,197
1,1.0,28.0,0.0,0.0,35.5000,0.0,1.0,0.0,0.0,1.0,1,24
2,3.0,19.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0,1,284
3,1.0,19.0,1.0,0.0,53.1000,0.0,1.0,0.0,0.0,1.0,0,749
4,3.0,35.9,0.0,0.0,8.4583,0.0,1.0,0.0,1.0,0.0,0,6


In [29]:
X_train.isna().sum()

Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
Survived       0
PassengerId    0
dtype: int64

In [30]:
X_train.to_csv("./data/cleaned/df_train.csv", index=False)

In [31]:
X_test["Survived"] = y_test
X_test["PassengerId"] = X_test_PassengerId
X_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived,PassengerId
0,3.0,29.0,0.0,0.0,7.8958,0.0,1.0,0.0,0.0,1.0,0,602
1,2.0,46.0,0.0,0.0,26.0000,0.0,1.0,0.0,0.0,1.0,0,398
2,1.0,26.0,0.0,0.0,78.8500,1.0,0.0,0.0,0.0,1.0,1,291
3,3.0,65.0,0.0,0.0,7.7500,0.0,1.0,0.0,1.0,0.0,0,281
4,1.0,61.0,0.0,0.0,33.5000,0.0,1.0,0.0,0.0,1.0,0,171


In [32]:
X_test.isna().sum()

Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
Survived       0
PassengerId    0
dtype: int64

In [33]:
X_test.to_csv("./data/cleaned/df_test.csv", index=False)